In [ ]:
import pandas as pd
import glob
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import numpy as np

path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016'], axis=1, inplace=True)
print(merged_df.columns)
merged_df
for col in merged_df.columns:
    if col not in ['residence_type', 'year']:
        merged_df[col] = merged_df[col].astype(float)
merged_df['year'] = merged_df['year'].astype(int)
#pd.get_dummies(merged_df['sa2_2021'])
merged_df.rename({'gdp(USD Millioins)': 'gdp', 'saving_rate(% of GDP)': 'saving_rate'}, axis=1, inplace=True)
merged_df['residence_type'] = merged_df['residence_type'].astype('category')
merged_df['residence_type'] = merged_df['residence_type'].cat.codes
merged_df.dropna(inplace=True)

y = merged_df['weekly_rent']
#merged_df.drop(['weekly_rent', 'year'], axis=1, inplace=True)
merged_df.drop('weekly_rent', axis=1, inplace=True)
X = merged_df
print(X)
sel = RandomForestRegressor(n_estimators = 100)
sel.fit(X, y)

In [ ]:
import os

# Directory
directory = "random_forest_pred"
  
# Parent Directory path
parent_dir = "../data/curated/"
# Path
path = os.path.join(parent_dir, directory)

# Create the directory
os.mkdir(path)

In [ ]:
# Predict for the next 5 years
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

# Check the accuracy before predicting with the trained random forest regressor
# Predict for the next 5 years
parent_dir = "../data/curated/2023_2027_data"

for filename in os.listdir(parent_dir):
    merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

    # Extract year from the file name 
    which_year = re.findall(r'\d+', filename)

    # Add year column to the dataset to fit the input into the model
    merged_df_yr = merged_df_yr.withColumn("year", lit(which_year[0]))

    for c in merged_df_yr.columns:
        if c not in ["year", "sa2_2021"]:
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 
        elif c in ["year", "sa2_2021"]:
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(IntegerType())) 

    merged_df_yr = merged_df_yr.toPandas()
    print(merged_df_yr["sa2_2021"])
    merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes
    merged_df_yr.dropna(inplace=True)

    # Reorder the columns 
    merged_df_yr_reordered = merged_df_yr[['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar',
       'min_distance_to_cbd', 'min_distance_to_park', 'min_distance_to_prim',
       'min_distance_to_second', 'min_distance_to_train',
       'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop',
       'gdp', 'saving_rate', 'income_per_person', 'population_density',
       'crime_cases']]

    # Predict with random forest tree
    prediction = sel.predict(merged_df_yr_reordered)

    new_csv_name = "../data/curated/random_forest_pred/" + filename
    print(merged_df_yr_reordered['sa2_2021'].astype(int))
    data = {'year': merged_df_yr_reordered['year'],
            'sa2_2021': merged_df_yr_reordered['sa2_2021'],
            'predicted_price': prediction }
    df = pd.DataFrame(data)
    print(df)
    df.to_csv(new_csv_name)